In [1]:
import ast
import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim

from copy import deepcopy
from sklearn.model_selection import KFold

In [2]:
class MLP_500_500_500_classifier(nn.Module):
    def __init__(self, input_len):
        super().__init__()

        self.layer1 = nn.Linear(input_len, 500)  
        self.relu = nn.ReLU()                          
        self.hidden_layer_relu = nn.Sequential(
            nn.Linear(500,500),
            nn.ReLU(),
            nn.Linear(500,500),
            nn.ReLU(),
            nn.Linear(500,500),
            nn.ReLU()
        )
        self.layer2 = nn.Linear(500, 30)

    def forward(self, x):

        x = self.relu(self.layer1(x))
        x = self.hidden_layer_relu(x)
        logits = self.layer2(x)
        
        return logits

In [3]:
# abrindo os dados de treinamento
df = pd.read_csv("./../ansatz_result/data.csv")
X = df.drop(columns=["target"]).to_numpy()
y = pd.DataFrame(df['target'].apply(ast.literal_eval).tolist()).to_numpy()

y_best_ansatz = np.argmax(y, axis=1)  

# criando o kfold
kf = KFold(n_splits=3, shuffle=True, random_state=42)

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {device} device")

Using cpu device


In [5]:
# implementação de um critério de parada para o modelo parar no ponto "ótimo"
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = float('inf')
        self.early_stop = False
        
    def __call__(self, val_loss):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

In [6]:
# 3. Função para treinar um fold
def train_fold(model, train_loader, val_loader, criterion, optimizer, n_epochs=1000000, patience = 10, min_delta = 0.0):
    model.to(device)
    early_stopping = EarlyStopping(patience=patience, min_delta=min_delta)
    best_model = None
    best_acc = 0.0
    
    for epoch in range(n_epochs):
        # Treinamento
        model.train()
        train_loss = 0.0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_batch = y_batch.squeeze()  # Converte (batch_size, 1) para (batch_size,)
            
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        # Validação
        model.eval()
        with torch.no_grad():
            val_loss = 0.0
            correct = 0
            total = 0
            for X_val, y_val in val_loader:
                X_val, y_val = X_val.to(device), y_val.to(device)
                y_val = y_val.squeeze()  # Garante que y_val é 1D
                
                outputs = model(X_val)
                val_loss += criterion(outputs, y_val).item()  
                _, predicted = torch.max(outputs.data, 1)
                correct += (predicted == y_val).sum().item()
                total += y_val.size(0)
        
        
        val_acc = correct / total
        val_loss /= len(val_loader)
        train_loss /= len(train_loader)
        
        print(f'Epoch {epoch+1}/{n_epochs} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f} - Val Acc: {val_acc:.4f}')
        
        # Early Stopping
        early_stopping(val_loss)
        if early_stopping.early_stop:
            print(f"Early stopping at epoch {epoch+1}")
            break
            
        # Salvar melhor modelo
        if val_acc > best_acc:
            best_acc = val_acc
            best_model = deepcopy(model.state_dict())
    
    return best_model, best_acc

In [7]:
results = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    print(f"\n=== Fold {fold + 1}/{3} ===")
    
    # Dividir e preparar dados
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y_best_ansatz[train_idx], y_best_ansatz[val_idx]
    
    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.LongTensor(y_train))
    val_dataset = TensorDataset(torch.FloatTensor(X_val), torch.LongTensor(y_val))
    
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32)
    
    # Inicializar modelo e otimizador
    model = MLP_500_500_500_classifier(22)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)  
    
    # Treinar fold
    best_model, best_acc = train_fold(model, train_loader, val_loader, criterion, optimizer, patience=15)
    results.append(best_acc)
    
    # Salvar modelo se necessário
    torch.save(best_model, f'./models_salvos/best_model_MLP-500-500-500-classifier_fold{fold+1}.pt')

# 7. Resultados finais
print("\n=== Resultados ===")
print(f"Acurácia média: {np.mean(results):.4f} ± {np.std(results):.4f}")
print(f"Acurácias por fold: {results}")


=== Fold 1/3 ===
Epoch 1/1000000 - Train Loss: 3.3278 - Val Loss: 3.0967 - Val Acc: 0.0917
Epoch 2/1000000 - Train Loss: 2.6295 - Val Loss: 2.7318 - Val Acc: 0.3000
Epoch 3/1000000 - Train Loss: 2.3866 - Val Loss: 2.7594 - Val Acc: 0.2583
Epoch 4/1000000 - Train Loss: 2.3590 - Val Loss: 2.9409 - Val Acc: 0.2583
Epoch 5/1000000 - Train Loss: 2.3672 - Val Loss: 2.7398 - Val Acc: 0.2583
Epoch 6/1000000 - Train Loss: 2.3165 - Val Loss: 2.6827 - Val Acc: 0.2750
Epoch 7/1000000 - Train Loss: 2.2520 - Val Loss: 2.7636 - Val Acc: 0.2583
Epoch 8/1000000 - Train Loss: 2.2765 - Val Loss: 2.7589 - Val Acc: 0.2583
Epoch 9/1000000 - Train Loss: 2.2673 - Val Loss: 2.7112 - Val Acc: 0.2750
Epoch 10/1000000 - Train Loss: 2.2143 - Val Loss: 2.7144 - Val Acc: 0.2667
Epoch 11/1000000 - Train Loss: 2.1834 - Val Loss: 2.7865 - Val Acc: 0.2750
Epoch 12/1000000 - Train Loss: 2.1720 - Val Loss: 2.9269 - Val Acc: 0.3083
Epoch 13/1000000 - Train Loss: 2.2330 - Val Loss: 2.7695 - Val Acc: 0.2583
Epoch 14/1000000

In [8]:
model = MLP_500_500_500_classifier(22).to(device)
#carregando o melhor fold treinado
model.load_state_dict(torch.load('.//models_salvos/best_model_MLP-500-500-500-classifier_fold2.pt'))  # Substitua pelo caminho correto
model.eval()  # Modo de avaliação

MLP_500_500_500_classifier(
  (layer1): Linear(in_features=22, out_features=500, bias=True)
  (relu): ReLU()
  (hidden_layer_relu): Sequential(
    (0): Linear(in_features=500, out_features=500, bias=True)
    (1): ReLU()
    (2): Linear(in_features=500, out_features=500, bias=True)
    (3): ReLU()
    (4): Linear(in_features=500, out_features=500, bias=True)
    (5): ReLU()
  )
  (layer2): Linear(in_features=500, out_features=30, bias=True)
)

In [9]:
input = X[6]
input_tensor = torch.FloatTensor(input).to(device)

with torch.no_grad():
    logits = model(input_tensor)
    print(logits)
    print(logits.shape)
    probabilities = torch.softmax(logits, dim=0)  # Shape (1, 30)

tensor([  1.9310,  -2.8520,   1.5080,  -4.5539, -12.4744, -12.4945,   1.6156,
          1.4059,  -3.2365,  -5.4026,  -4.2948,   0.5748,   1.0659,   0.8931,
          1.3808, -12.2624,  -1.5550,  -4.8942,  -7.2296,  -5.5795,  -3.8493,
          0.5045,  -3.6012,  -5.8621,  -4.7726,  -1.7680,  -7.1251,  -4.2433,
         -7.4999,  -3.9068])
torch.Size([30])


In [10]:
print(probabilities)

tensor([2.0346e-01, 1.7030e-03, 1.3328e-01, 3.1052e-04, 1.1279e-07, 1.1055e-07,
        1.4842e-01, 1.2034e-01, 1.1594e-03, 1.3289e-04, 4.0237e-04, 5.2414e-02,
        8.5654e-02, 7.2061e-02, 1.1736e-01, 1.3943e-07, 6.2303e-03, 2.2097e-04,
        2.1382e-05, 1.1135e-04, 6.2823e-04, 4.8857e-02, 8.0513e-04, 8.3941e-05,
        2.4953e-04, 5.0350e-03, 2.3739e-05, 4.2365e-04, 1.6319e-05, 5.9312e-04])


In [11]:
# Obter índices e valores das top-k probabilidades (ex.: top-5)
k = 3
top_k_probs, top_k_indices = torch.topk(probabilities, k=k, dim=0)

# Converter para numpy e exibir
top_k_probs = top_k_probs.cpu().numpy().flatten()
top_k_indices = top_k_indices.cpu().numpy().flatten()

print("Melhores ansatzes e suas probabilidades:")
for i, (idx, prob) in enumerate(zip(top_k_indices, top_k_probs)):
    print(f"{i+1}º: Ansatz {idx} - Probabilidade: {prob:.4f}")

Melhores ansatzes e suas probabilidades:
1º: Ansatz 0 - Probabilidade: 0.2035
2º: Ansatz 6 - Probabilidade: 0.1484
3º: Ansatz 2 - Probabilidade: 0.1333
